# Chapter 7 - Eligibility Traces

This chapter starts by introducing n-step TD methods, which sit between the two extremes: TD(0), or 1-step TD and Monte-Carlo (which can be considered an $\infty$-step TD). Then it builds on that to consider n-step backups and n-step returns, and mixing multiple n-step returns into a complex backup, which is the TD($\lambda$) algorithm.

It then looks at the TD($\lambda$) family of algorithms from two perspectives - forward and backward view, by using forward for a better conceptual understanding, while the backward view is more appropriate for the actual implementation.

And finally we are introduced to variants of known algorithms that use the TD($\lambda$) updates, like Sarsa($\lambda$), Q($\lambda$) and Eligibility Traces for actor-critic methods.

### Exercise 7.1

Why do you think a larger random walk task (19 states instead of 5) was used in the examples of this chapter? Would a smaller walk have shifted the advantage to a different value of $n$? How about the change in left-side outcome from $0$ to $-1$? Would that have made any difference in the best value of $n$?

A random walk over a small number of states should see not much difference between using 1-step TD updates or using every-visit MC, because the probability of reaching a terminal state from any given state is relatively high, e.g. between $\frac{1}{2}$ and $\frac{1}{8}$ for the 5 states task. That means that a small proportion of epsisodes would actually use a full n-step backup for the updates, i.e. after some (small) optimal length $n^*$, the value of longer $n$ would bring no benefit. Therefore a random walk task with 19 states gives more "opportunity" to show how different outcomes can be for different $n$'s, and how the optimum is somewhere in the middle between 1-step TD and MC.

For a similar reason, the change of the left terminal reward to $-1$ provides a more "dramatic" curve between multiple values of $n$; intuitively I expect the curves would have been flatter if the left terminal reward remained $0$, especially over only 10 episodes to learn from.

### Exercise 7.2

Why do you think on-line methods worked better than off-line methods on the example taks?

I expect the off-line methods have a slower start on most tasks, because they only update once per episode. In this case the results presented and the graphs in Figure 7.2 used only the first 10 episodes, which can be considered little for a task with 19 states! However, it must be mentioned that the advantage of off-line methods is that the update is usually more stable, being an average over potentially multiple passes through a state per episode and on the longer term converge to the same optimum as the on-line methods.

### Exercise 7.3

Explain why the plots for off-line n-step TD show a worse performance for $n=3, 5, 7, 9$ at a much lower value of $\alpha$ than for $n=2, 4, 6, etc.$

First read the [Other Notes on the Errata page](http://incompleteideas.net/book/first/errata.html). <br>

My intuition goes something like this. In this task all rewards are $0$ except for the terminal transitions. So for all n-step backups starting from $s_t$, since $\gamma=1$, the off-line update only depends on the sum of the values of the end state(s) $s_{t+n}$ (of which some can be terminal):

$$
V(s_t) \leftarrow V(s_t) + \alpha \sum_{s_{t+n}} \left[ V(s_{t+n}) - V(s_t) \right]
$$

So $V(s_t)$ is incremented slightly towards the sum of the n-step TD errors, nothing strange about that. However, these TD errors depend only on the value of the states in which each n-step jumps finish.

The second observation is that by starting from any fixed position and following n-step jumps, it is easy to notice that if $n$ is odd, the end positions will always have the opposite parity of the starting position (excepting terminal states). For example from state $7$, all possible 3-step jumps finish in either $4$, $6$, $8$ or $10$. But if $n$ is even, the end state always has the same parity as the starting state. For instance from $7$ all possible 4-step jumps finish in either $3$, $5$, $7$, $9$ or $11$.

From this we draw two consequences:

 - If $n$ is odd, after every off-line update as per the formula above, the values of $V(odd\_positions)$ will be updated towards some function of $V(even\_positions)$ and vice-versa. Which can create a self-reinforcing error propagation from one update to the next over multiple episodes, especially for the positions far from the two terminal states. This effect is magnified by a large $\alpha$.
 
 - If $n$ is even, a good proportion of n-steps will finish in the starting position, thus reducing the update step, because the n-step TD error will be zero for those cases (as per the formula). It could be considered equivalent to using a smaller $\alpha$ overall. Also, $V(odd\_positions)$ will be updated only from other $V(odd\_positions)$ and similarly for even positions. This kind of "segregation of state values" means that if there is a large over/under-estimate somewhere, it will get propagated to at most half of the states around it, not to all.

For these two reasons we observe worse performance from odd $n$'s, especially for larger values of $\alpha$ and small number of episodes.